### Importing modules, downloading data and applying regex on it

In [8]:
import requests, re
import pandas as pd

response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
save = re.findall("<td>(.*?)\n*</td>", response)

### Colecting the data from regex above and saving it (in a dict and in a list)

In [9]:
i = 0
postcodes = {}
post_lines = []
p = re.compile("^\w\d\w$")
for line in save:
    if p.search (line) != None:
        postcodes[i] = []
        i_old = i
        i+=1
        post_lines.append(line)
    elif "title" in line:
        postcodes[i_old].append(re.findall('>(.*?)<', line)[0])
    elif i != i_old:
        postcodes[i_old].append(line)


### Deleting rows with "Not assigned" on it.

In [10]:
borough_neigh = []
for i in range(len(postcodes)):
    borough_neigh.append(postcodes[i])

PostalCode = []
Borough = []
Neigh = []

for i in range(len(borough_neigh)):
    if "Not assigned" not in borough_neigh[i]:
        Borough.append(borough_neigh[i][0])
        Neigh.append(borough_neigh[i][1:])
        PostalCode.append(post_lines[i])

### Creating a dictionary that will be used for the final creation of the three columns of the dataframe

In [11]:
dict_p = {}
for i in range(len(PostalCode)):
    dict_p[PostalCode[i]] = []

for i in range(len(PostalCode)):
    if Borough[i] not in dict_p[PostalCode[i]]:
        dict_p[PostalCode[i]].append(Borough[i])
    dict_p[PostalCode[i]].append(Neigh[i])

### Creating the three columns

In [12]:
Final_Postal_Code = []
Final_Borough = []
Neigh = []

for i in dict_p:
    Final_Postal_Code.append(i)
    Final_Borough.append(dict_p[i][0])
    Neigh.append(dict_p[i][1:])

### Converting data in column Neighborhood to be separated with commas

In [13]:
Final_Neighborhood = []

final_str = ""
for i in range(len(Neigh)):
    for j in Neigh[i]:
        final_str += j[0]+", "

    Final_Neighborhood.append(final_str[:-2])
    final_str = ""

### Create the dataframe as required in the lab

In [14]:
dict_to_dataframe = {'PostalCode': Final_Postal_Code, 'Borough': Final_Borough, 'Neighborhood': Final_Neighborhood}
df = pd.DataFrame(dict_to_dataframe)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### Showing the shape of the dataframe

In [15]:
df.shape

(103, 3)